### Scraping fighters data

In [2]:
import requests,bs4

In [12]:
BASE_URL = "http://ufcstats.com/statistics/fighters"

In [67]:
import string

# The page is organized by fighters names so I decided to
# Loop through different pages using an alphabets list
alphabets = list(string.ascii_lowercase)
for letter in alphabets:
    fighters_DOM_per_letter = requests.get(f"{BASE_URL}?char={letter}")
    soup = bs4.BeautifulSoup(fighters_DOM_per_letter.text, 'html.parser')
    pagination = soup.find('ul', class_='b-statistics__paginate')
    n_pages = len(pagination.find_all('li'))

    # Initialize the fighters dict
    if letter == 'a':
        headers = [th.get_text(strip=True) for th in soup.select("table thead th")]
        fighters_data = {header: [] for header in headers}
    
    # Not adding 1 to the number of list items because
    # there is a link for "all"
    for i in range(1, n_pages):
        current_page = requests.get(f"{BASE_URL}?char={letter}&page={i}")
        soup_1 = bs4.BeautifulSoup(current_page.text, 'html.parser')

        # Scraping fighters tabular data 
        for row in soup_1.select("table tbody tr"):
            cells = row.find_all('td',class_='b-statistics__table-col')
            
            if len(cells) == 0 :
                continue
            
            while len(cells) < len(headers):
                cells.append(None)
            
            for header,cell in zip(headers,cells):
                if cell:
                    fighters_data[header].append(cell.get_text(strip=True))
                else :
                    fighters_data[header].append(None)

In [68]:
import pandas as pd

df = pd.DataFrame(fighters_data)

In [69]:
df.head(30)

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,Tom,Aaron,,--,155 lbs.,--,,5,3,0,
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,
3,Darion,Abbey,,"6' 2""",265 lbs.,"80.0""",Orthodox,9,5,0,
4,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0,
5,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,6,1,0,
6,Mansur,Abdul-Malik,,"6' 2""",185 lbs.,"80.0""",Orthodox,8,0,1,
7,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20,8,0,
8,Hiroyuki,Abe,Abe Ani,"5' 6""",145 lbs.,--,Orthodox,8,15,3,
9,Daichi,Abe,,"5' 11""",170 lbs.,"71.0""",Orthodox,6,2,0,
